# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # skip the .ipynb_checkpoints files created when we open a file in workspace UI
    if '.ipynb_checkpoints' in root:
        continue

    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*csv'))
    print(len(file_path_list))
    # Uncomment below to see all the files
    #print(file_path_list)

/home/workspace
30


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
    print("Reading file {}".format(f))

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows
total_data_rows = len(full_data_rows_list)
print("Total rows from all input files {}".format(total_data_rows))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

Reading file /home/workspace/event_data/2018-11-30-events.csv
Reading file /home/workspace/event_data/2018-11-23-events.csv
Reading file /home/workspace/event_data/2018-11-22-events.csv
Reading file /home/workspace/event_data/2018-11-29-events.csv
Reading file /home/workspace/event_data/2018-11-11-events.csv
Reading file /home/workspace/event_data/2018-11-14-events.csv
Reading file /home/workspace/event_data/2018-11-20-events.csv
Reading file /home/workspace/event_data/2018-11-15-events.csv
Reading file /home/workspace/event_data/2018-11-05-events.csv
Reading file /home/workspace/event_data/2018-11-28-events.csv
Reading file /home/workspace/event_data/2018-11-25-events.csv
Reading file /home/workspace/event_data/2018-11-16-events.csv
Reading file /home/workspace/event_data/2018-11-18-events.csv
Reading file /home/workspace/event_data/2018-11-24-events.csv
Reading file /home/workspace/event_data/2018-11-04-events.csv
Reading file /home/workspace/event_data/2018-11-19-events.csv
Reading 

In [4]:
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables

# keep track of empty artist rows
empty_artist=0

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        # skip row with empty artist info
        if (row[0] == ''):
            empty_artist+=1
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
        
    print("Rows with empty artist info {}".format(empty_artist))

Rows with empty artist info 1236


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    all_rows = sum(1 for line in f)

print("Rows written in result file {}".format(all_rows-1))
print("Total rows from all input files {}".format(total_data_rows))
print("Rows in input files with empty artist info {}".format(empty_artist))
print("Expected rows to be written in result file {}".format(total_data_rows-empty_artist))

Rows written in result file 6820
Total rows from all input files 8056
Rows in input files with empty artist info 1236
Expected rows to be written in result file 6820


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [7]:
# Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION =
    {'class' : 'SimpleStrategy', 'replication_factor' :1 }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
'''
Table 1: Should have partition key as sessionId and clustering column as 
itemInSession so they can be used in WHERE clause.
'''
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS music_library1 "
query = query + "(sessionId int, itemInSession int, artist text, \
        song_title text, song_length float, \
        PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [10]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## INSERT statements into the `query` variable
        query = "INSERT INTO music_library1 (sessionId, itemInSession, \
                artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## Column element that should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, we would change the code below to `line[0], line[1]`
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
## SELECT statement to verify the data was entered into the table
query = "SELECT artist, song_title, song_length from music_library1 WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, "\t", row.song_title, "\t", row.song_length)

Faithless 	 Music Matters (Mark Knight Dub) 	 495.30731201171875


In [12]:
'''
Table 2: Should have partition key as (userid, sessionid) and clustering
column as itemInSession so they can be used in WHERE clause and song can be
sorted by itemInSession column.
'''

## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS music_library2 "
query = query + "(userid int, sessionid int, itemInSession int, \
        song_title text, artist text, user text, \
        PRIMARY KEY ((userid, sessionid), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [13]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## INSERT statements into the `query` variable
        query = "INSERT INTO music_library2 (userid, sessionid, itemInSession, song_title, artist, user)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        ## Column element that should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, we would change the code below to `line[0], line[1]`
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[9], line[0], line[1] + " " + line[4]))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [14]:
## SELECT statement to verify the data was entered into the table
query = "SELECT song_title, artist, user from music_library2 WHERE userid = 10 AND sessionid = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.song_title, "\t", row.artist, "\t", row.user)

Keep On Keepin' On 	 Down To The Bone 	 Sylvie Cruz
Greece 2000 	 Three Drives 	 Sylvie Cruz
Kilometer 	 Sebastien Tellier 	 Sylvie Cruz
Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 	 Lonnie Gordon 	 Sylvie Cruz


In [15]:
'''
Table 3: Should have partition key as song so it can be used in WHERE clause. I have artist column as 
I have userid as clustering column to make the Primary Key UNIQUE.
'''

## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "CREATE TABLE IF NOT EXISTS music_library3 "
query = query + "(song text, userid int, user text, \
        PRIMARY KEY (song, userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [16]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## INSERT statements into the `query` variable
        query = "INSERT INTO music_library3 (song, userid, user)"
        query = query + " VALUES (%s, %s, %s)"
        ## Column element that should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, we would change the code below to `line[0], line[1]`
        try:
            session.execute(query, (line[9], int(line[10]), line[1] + " " + line[4]))
        except Exception as e:
            print(e)

In [17]:
## SELECT statement to verify the data was entered into the table
query = "SELECT user from music_library3 WHERE song='All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    #print(row)
    print(row.user)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [18]:
## Drop the table before closing out the sessions
query1 = "DROP TABLE music_library1"
try:
    session.execute(query1)
except Exception as e:
    print(e)

query2 = "DROP TABLE music_library2"
try:
    session.execute(query2)
except Exception as e:
    print(e)

query3 = "DROP TABLE music_library3"
try:
    session.execute(query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()